In [ ]:
# %%capture
# !pip install utm
# !pip install openpyxl

In [3]:
import numpy as np
import pandas as pd
import seaborn as sns
from matplotlib import cm
import torch
import torch.nn as nn
import torch.nn.functional as F
from sklearn.gaussian_process.kernels import RBF
from sklearn.model_selection import train_test_split
from torch.utils.data import Dataset, DataLoader
from sklearn.preprocessing import StandardScaler, MinMaxScaler
from sklearn.gaussian_process import GaussianProcessRegressor
import matplotlib.pyplot as plt
import utm

import warnings
warnings.filterwarnings("ignore")

In [4]:
data = pd.read_excel("../Dataset/Maharashtra_Soil_Nutrients_Data.xlsx")
data.head()

,lon,lat,OC,N,P,K
0,73.401111,17.894722,1.08,756.0,9.43,834.37
1,73.401389,17.894722,1.12,781.2,9.21,265.10
2,73.402222,17.894722,0.68,478.8,8.99,318.96
3,73.403056,17.894722,1.76,1234.8,9.65,954.77
4,73.403333,17.894722,1.78,1247.4,8.77,371.77


In [5]:
def scaled_coord(x,y):
    """
    parameters
    ----------
    x : numpy array, float64
        list of longitude cordinates
    y : numpy array, float64
        list of latitude cordinates
        
    return
    ------
    scaled(0-1) x and y
    """
    x = (x-x.min())/(x.max()-x.min())
    y = (y-y.min())/(y.max()-y.min())
    return x,y

In [6]:
%%time

val_col = ['OC','N','P','K']
values = data[val_col]
coordinates = data[['lon','lat']]
#lat,lon to utm projection

x,y,zone,ut = utm.from_latlon(coordinates['lat'].values,coordinates['lon'].values)

lon,lat = y/1000,x/1000 #in km

# lon, lat = scaled_coord(lon,lat)
# normalize values of OC, N, K, P

#standardise lon and lat
# lon = (lon-np.mean(lon))/np.std(lon)
# lat = (lat-np.mean(lat))/np.std(lat)

test_k = MinMaxScaler().fit_transform(values)
values = test_k

Wall time: 15 ms


In [7]:
data['lon'] = lon
data['lat'] = lat
for i,col in enumerate(val_col):
    data[col] = values[:,i]

In [8]:
data.head()

,lon,lat,OC,N,P,K
0,1979.264401,330.619321,0.011632,0.079661,0.001163,0.012262
1,1979.264149,330.648754,0.012067,0.082316,0.001136,0.003893
2,1979.263392,330.737053,0.007283,0.050450,0.001109,0.004684
3,1979.262635,330.825353,0.019024,0.130115,0.001190,0.014032
4,1979.262383,330.854785,0.019241,0.131443,0.001082,0.005461


In [9]:
#split dataset into train and test
# split the dataset into train and test dataset
ix = np.random.choice(data.shape[0],int(data.shape[0]*0.2),replace = False)
data_train = data.iloc[[int(i) for i in range(data.shape[0]) if i not in ix]].reset_index(drop = True)
data_test = data.iloc[ix].reset_index(drop = True)

In [10]:
data_train.shape, data_test.shape

((20837, 6), (5209, 6))

In [11]:
data_train.head()

,lon,lat,OC,N,P,K
0,1979.264149,330.648754,0.012067,0.082316,0.001136,0.003893
1,1979.263392,330.737053,0.007283,0.050450,0.001109,0.004684
2,1979.262635,330.825353,0.019024,0.130115,0.001190,0.014032
3,1979.261879,330.913651,0.015110,0.103560,0.001109,0.010119
4,1979.261627,330.943085,0.016306,0.111527,0.001136,0.007821


In [12]:
data_test.head()

,lon,lat,OC,N,P,K
0,2018.770731,429.996503,0.005435,0.010744,0.000575,0.005802
1,2012.560689,491.570067,0.004783,0.066383,0.001542,0.010198
2,1994.453146,558.712577,0.003805,0.026551,0.001535,0.005758
3,2011.329738,497.238802,0.003479,0.024338,0.001752,0.012507
4,2145.359516,530.740139,0.004674,0.008520,0.000761,0.003950


In [13]:
data_test.shape

(5209, 6)

## Data loading in torch.Dataloader

In [14]:
class NutrientsDataset(Dataset):
    def __init__(self, df, num_context=40, num_extra_target=10):
        self.df = df
        self.num_context = num_context
        self.num_extra_target = num_extra_target

    def get_rows(self, i):
        rows = self.df.iloc[i : i + (self.num_context + self.num_extra_target)].copy()
        x = rows.iloc[:,:2].copy()
        y = rows.iloc[:,2:].copy()
        return x, y


    def __getitem__(self, i):
        x, y = self.get_rows(i)
        return x.values, y.values
        
    def __len__(self):
        return len(self.df) - (self.num_context + self.num_extra_target)

In [15]:
def npsample_batch(x, y, size=None, sort=False):
    
    """Sample from numpy arrays along 2nd dim."""
    inds = np.random.choice(range(x.shape[1]), size=size, replace=False)
    return x[:, inds], y[:, inds]

def collate_fns(max_num_context, max_num_extra_target, sample, sort=True, context_in_target=True):
    def collate_fn(batch, sample=sample):
        # Collate
        x = np.stack([x for x, y in batch], 0)
        y = np.stack([y for x, y in batch], 0)

        # Sample a subset of random size
        num_context = np.random.randint(4, max_num_context)
        num_extra_target = np.random.randint(4, max_num_extra_target)

        x = torch.from_numpy(x).float()
        y = torch.from_numpy(y).float()

        
        x_context = x[:, :max_num_context]
        y_context = y[:, :max_num_context]
    
        x_target_extra = x[:, max_num_context:]
        y_target_extra = y[:, max_num_context:]
        
        if sample:

            x_context, y_context = npsample_batch(
                x_context, y_context, size=num_context
            )

            x_target_extra, y_target_extra = npsample_batch(
                x_target_extra, y_target_extra, size=num_extra_target, sort=sort
            )

        # do we want to compute loss over context+target_extra, or focus in on only target_extra?
        if context_in_target:
            x_target = torch.cat([x_context, x_target_extra], 1)
            y_target = torch.cat([y_context, y_target_extra], 1)
        else:
            x_target = x_target_extra
            y_target = y_target_extra

        
        return x_context, y_context, x_target, y_target

    return collate_fn

 ## NP Model

In [16]:
class baseNPBlock(nn.Module):
    """relu non-linearities for NP block"""
    def __init__(self, inp_size,op_size, norm, bias = False, p = 0):
        """init function for linear2d class
        
        parameters
        ----------
        inp_size : int
                input dimension for the Encoder part (d_in)
        op_size : int
                output dimension for Encoder part(d_out)
        norm : str
                normalization to be applied on linear output
                pass norm == 'batch' to apply batch normalization
                else dropout normalization is applied
        bias : bool
                if True, bias is included for linear layer else discarded
        p : float
                probality to be considered while applying Dropout regularization
                
        """
        super().__init__()
        self.norm = norm
        self.linear = nn.Linear(inp_size,op_size,bias = bias)
        self.relu  = nn.ReLU()
        self.batch_norm = nn.BatchNorm2d(op_size)
        self.dropout = nn.Dropout2d(p)
        
    def forward(self,x):
        x = self.linear(x)
        x = self.batch_norm(x.permute(0,2,1)[:,:,:,None]) if self.norm == 'batch' else self.dropout(x.permute(0,2,1)[:,:,:,None])
        
        x = self.relu(x[:,:,:,0].permute(0,2,1))
        return x

In [17]:
class batch_MLP(nn.Module):
    """ Batch MLP layer for NP-Encoder"""
    def __init__(self, in_size, op_size, num_layers, norm, p = 0):
        """init function for linear2d class
        
        parameters
        ----------
        inp_size : int
                input dimension for the Encoder part (d_in)
        op_size : int
                output dimension for Encoder part(d_out)
        norm : str
                normalization to be applied on linear output
                pass norm == 'batch' to apply batch normalization
                else dropout normalization is applied
                
        return torch.tensor of size (B,num_context_points,d_out)
        """
        super().__init__()
        self.in_size = in_size
        self.op_size = op_size
        self.num_layers = num_layers
        self.norm  = norm
        
        self.first_layer = baseNPBlock(in_size, op_size, self.norm, False,p)
        self.encoder = nn.Sequential(*[baseNPBlock(op_size, op_size, self.norm, False, p) for layer in range(self.num_layers-2)])
        self.last_layer = nn.ReLU()
        
    def forward(self, x):
        x = self.first_layer(x)
        x = self.encoder(x)
        x = self.last_layer(x)
        
        return x

In [18]:
class LinearAttention(nn.Module):
    def __init__(self,in_ch, out_ch):
        super().__init__()
        self.linear = nn.Linear(in_ch, out_ch, bias = False)
        torch.nn.init.normal_(self.linear.weight,std = in_ch**0.5) #initilize weight matrix
        
    def forward(self,x):
        return self.linear(x)
    
    
class AttentionModule(nn.Module):
    def __init__(
        self,
        hidden_dim, 
        attn_type , 
        attn_layers,
        x_dim, 
        rep='mlp',
        n_multiheads = 8,
        norm = 'dropout',
        p = 0):
        
        super().__init__()
        self.rep = rep
#         self.n_multiheads = n_multiheads
        # rep determines whether raw input given to the model would be used as key and query or
        # it's output through MLP. 
        if self.rep =='mlp':
            
            #Both Key and Value needs to have same dimension
            self.batch_mlpk = batch_MLP(x_dim, hidden_dim, attn_layers, norm ,p)
            self.batch_mlpq = batch_MLP(x_dim, hidden_dim, attn_layers, norm, p)
        
        
        if attn_type == 'uniform':
            self.attn_func = self.uniform_attn
        if attn_type=='laplace':
            self.attn_func = self.laplace_attn
        if attn_type == 'dot':
            self.attn_func = self.dot_attn
        elif attn_type == 'multihead':
            self.w_k = nn.ModuleList([LinearAttention(hidden_dim,hidden_dim) for head in range(n_multiheads)])
            self.w_v = nn.ModuleList([LinearAttention(hidden_dim,hidden_dim) for head in range(n_multiheads)])
            self.w_q = nn.ModuleList([LinearAttention(hidden_dim,hidden_dim) for head in range(n_multiheads)])
            
            self.w = LinearAttention(hidden_dim*n_multiheads,hidden_dim)
            self.attn_func = self.multihead_attn
            self.num_heads = n_multiheads
            
            
            
    def forward(self, k, q, v):
        if self.rep =='mlp':
            k = self.batch_mlpk(k) #(B, n, H)
            q = self.batch_mlpq(q) #(B, m, H)
        
        rep = self.attn_func(k,q,v)
        
        return rep
    
    
    def uniform_attn(self, k, q, v):
        num_points = q.shape[1]
        rep = torch.mean(v, axis = 1, keepdim = True)
        rep = rep.repeat(1,num_points,1)
        
        return rep
    
    def laplace_attn(self, k, q, v, scale = 0.5):
        k = k.unsqueeze(1)
        v = v.unsqueeze(2)
        
        w = torch.abs((k-v)*scale)
        w = w.sum(dim = -1)
        weight = torch.softmax(w, dim = -1)
        
        #batch matrix multiplication (einstein summation convention for tensor)
        rep = torch.einsum("bik, bkj -> bij",weight, v)
        
        return rep
    
    
    def dot_product_attn(self, k, q, v):
#         print("k =",k.shape)
#         print("q =",q.shape)
#         print("v =",v.shape)    
        β = q.shape[-1]**0.5
        w_unnorm = torch.einsum('bjk,bik->bij', k, q)/β
#         print("w_unnorm =",w_unnorm.shape)
        
        weight = torch.softmax(w_unnorm, dim = -1)
        rep = torch.einsum("bik, bkj -> bij",weight, v)
#         print("rep =",rep.shape)
        return rep
    
    def multihead_attn(self, k , q, v):
        outs = []
        
        for i in range(self.num_heads):
            k = self.w_k[i](k) #(B, n, H)
#             print("k =",k.shape)
            q = self.w_q[i](q) #(B, m, H)
#             print("q =",q.shape)
            v = self.w_v[i](v) #(B, n, H)
#             print("v =",v.shape)
            out = self.dot_product_attn(k, q, v)
            outs.append(out)
            
        outs = torch.stack(outs, dim = -1) #(B, m, H, n_heads)
#         print("outs dim =", outs.shape)
        outs = outs.view(outs.shape[0], outs.shape[1], -1) #(B, m, n_heads*H)
#         print("outs shape =",outs.shape)
        rep = self.w(outs) #(B, m, H)
        
        return rep
    
    

In [19]:
# AttentionModule?

In [20]:
class DeterministicEncoder(nn.Module):
    def __init__(
                self,
                in_dim,
                x_dim,
                norm = 'dropout',
                hidden_dim = 32,
                encoder_layer = 2,
                self_attn_type ='dot',
                cross_attn_type ='dot',
                p_encoder = 0,
                p_attention = 0,
                attn_layers = 2,
                use_self_attn = False
                ):
        super().__init__()
        
        self.use_self_attn = use_self_attn
        
        self.encoder = batch_MLP(in_dim, hidden_dim, encoder_layer,norm, p_encoder)
        
        if self.use_self_attn:
            self.self_attn = AttentionModule(hidden_dim, self_attn_type, attn_layers,x_dim, rep = 'mlp',norm = norm, p = p_attention)
            
        self.cross_attn = AttentionModule(hidden_dim, cross_attn_type, attn_layers, x_dim)
        
    
    def forward(self, context_x, context_y, target_x):
        #concatenate context_x, context_y along the last dim.
        det_enc_in = torch.cat([context_x, context_y], dim = -1)
        
        det_encoded = self.encoder(det_enc_in) #(B, n, hd)
        
        if self.use_self_attn:
            det_encoded = self.self_attn(det_encoded, det_encoded, det_encoded)
            
        h = self.cross_attn(context_x, target_x, det_encoded)
        
        return h
        
        
    
        
        

In [21]:
class LatentEncoder(nn.Module):
    def __init__(self,
                in_dim,
                hidden_dim = 32,
                latent_dim = 32,
                self_attn_type = 'dot',
                encoder_layer = 3,
                min_std = 0.01,
                norm = 'dropout',
                p_encoder = 0,
                p_attn = 0,
                use_self_attn = False,
                attn_layers = 2,
                ):
        
        super().__init__()
        
        self._use_attn = use_self_attn
        
        self.encoder = batch_MLP(in_dim, hidden_dim, encoder_layer,norm, p_encoder)
        
        if self._use_attn:
            self.self_attn = AttentionModule(hidden_dim, self_attn_type, attn_layers,x_dim, rep = 'identity',norm = norm, p = p_attention)
        
        self.secondlast_layer = nn.Linear(hidden_dim, hidden_dim)
        self.mean = nn.Linear(hidden_dim, latent_dim)
        self.l_sigma = nn.Linear(hidden_dim, latent_dim) 
        self.min_std = min_std
#         self.use_lvar = use_lvar
        self.use_attn = use_self_attn
        
        
        
    def forward(self,x,y):
        encoder_inp = torch.cat([x,y], dim = -1) 
        
        encoded_op = self.encoder(encoder_inp)#(B, n, hd)
#         print("encoder_op shape = ",encoded_op.shape)
        if self.use_attn:
            encoded_op = self.self_attn(encoded_op, encoded_op, encoded_op) #(B, n, hd)
            
        
        mean_val = torch.mean(encoded_op, dim = 1) #mean aggregation (B, hd)
        
        #further MLP layer that maps parameters to gaussian latent
        mean_repr = torch.relu(self.secondlast_layer(mean_val)) #(B, hd)
        
        μ = self.mean(mean_repr) # (B, ld)
#         print("mean = ", μ.shape)
        log_scale = self.l_sigma(mean_repr) #(B, ld)
        
        #to avoid mode collapse
        σ = self.min_std + (1-self.min_std)*torch.sigmoid(log_scale*0.5) #(b, ld)
#         print(σ)
        dist = torch.distributions.Normal(μ, σ)
        
        return dist
        
        
            

In [22]:
class Decoder(nn.Module):
    def __init__(self,
                 x_dim,
                 y_dim,
                 hidden_dim = 32,
                 latent_dim = 32,
                 n_decoder_layer = 3,
                 use_deterministic_path = True,
                 min_std = 0.01,
                 norm = 'dropout',
                 dropout_p = 0,
                ):
        super().__init__()
        
        self.norm = norm
        self.target_transform = nn.Linear(x_dim, hidden_dim)
        
        if use_deterministic_path:
            hidden_dim_2 = 2 * hidden_dim + latent_dim
        else:
            hidden_dim_2 = hidden_dim + latent_dim
            
        self.decoder = batch_MLP(hidden_dim_2, hidden_dim_2, n_decoder_layer, norm, dropout_p)
        
        self.mean = nn.Linear(hidden_dim_2, y_dim)
        self.std = nn.Linear(hidden_dim_2, y_dim)
        self.deterministic_path = use_deterministic_path
        self.min_std = min_std
        
        
    def forward(self, r, z, t_x):
        x = self.target_transform(t_x)
        
        if self.deterministic_path:
            z = torch.cat([r,z], dim = -1)
#             print("z.shape =", z.shape)
        r = torch.cat([z,x], dim = -1)
        
        r = self.decoder(r)
        
        mean = self.mean(r)
        log_sigma = self.std(r)
        
        #clamp sigmad
        sigma = self.min_std + (1 - self.min_std) * F.softplus(log_sigma)
        
        dist = torch.distributions.Normal(mean,sigma)
        
        return dist

In [23]:
class LatentModel(nn.Module):
    def __init__(self,
               x_dim,
               y_dim,
               hidden_dim = 32,
               latent_dim = 32,
               latent_self_attn_type = 'multihead',
                det_self_attn_type = 'multihead',
                det_cross_attn_type = 'multihead',
               n_lat_enc_layer = 2,
               n_det_enc_layer = 2,
               n_decoder_layer = 2,
               use_deterministic_enc = False,
               min_std = 0.01,
               p_drop = 0,
               norm = 'dropout',
               p_attn_drop = 0,
               attn_layers = 2,
               use_self_attn = False,
               context_in_target = True,
                training = False):
        
        super().__init__()
        self.laten_encoder = LatentEncoder(x_dim+y_dim,
                                           hidden_dim=hidden_dim,
                                           latent_dim=latent_dim,
                                           self_attn_type=latent_self_attn_type,
                                           encoder_layer=n_lat_enc_layer,
                                           min_std=min_std,
                                           norm = norm,
                                           p_encoder=p_drop,
                                           p_attn=p_attn_drop,
                                           use_self_attn=use_self_attn,
                                           attn_layers=attn_layers 
                                          )
        self.deterministic_encoder = DeterministicEncoder(x_dim+y_dim,
                                                          x_dim,
                                                          norm = norm,
                                                          hidden_dim=hidden_dim,
                                                          encoder_layer=n_det_enc_layer,
                                                          self_attn_type=det_self_attn_type,
                                                          cross_attn_type=det_cross_attn_type,
                                                          p_encoder=p_drop,
                                                          p_attention=p_attn_drop,
                                                          attn_layers=attn_layers,
                                                          use_self_attn=use_self_attn
                                                         )
        self.decoder = Decoder(x_dim,
                              y_dim,
                              hidden_dim  = hidden_dim,
                              latent_dim=latent_dim,
                              n_decoder_layer=n_decoder_layer,
                              use_deterministic_path=use_deterministic_enc,
                              min_std=min_std,
                              norm=norm,
                              dropout_p=p_drop
                              )
        self.use_deterministic_enc = use_deterministic_enc
        self.context_in_target = context_in_target
        self.training = training
        
        
    def forward(self, c_x, c_y, t_x, t_y = None):
        dist_prior = self.laten_encoder(c_x, c_y)

        if t_y is not None:
            dist_posterior = self.laten_encoder(t_x, t_y)
            z = dist_posterior.loc
        else:
            z = dist_prior.loc
            
        n_target = t_x.shape[1]
        z = z.unsqueeze(1).repeat(1, n_target,1) #(B, n_target, L)
        
        if self.use_deterministic_enc:
            r = self.deterministic_encoder(c_x, c_y, t_x) #(B, n_target=m, H)
#             print(r.shape)
        else:
            r = None
            
        dist = self.decoder(r, z, t_x)
        
        #at test time, target y is not Known so we return None
        if t_y is not None:
            log_p = dist.log_prob(t_y).mean(-1)
            kl_loss = torch.distributions.kl_divergence(dist_posterior, dist_prior).mean(-1)
            kl_loss = kl_loss[:,None].expand(log_p.shape)
            loss = (kl_loss-log_p).mean()
            mse_loss = F.mse_loss(dist.loc, t_y, reduction = 'none')[:,:c_x.size(1)].mean()
        else:
            kl_loss  =None
            log_p = None
            mse_loss = None
            loss = None
            
        y_pred = dist.rsample() if self.training else dist.loc
            
        return y_pred,  dict(loss = loss, loss_p = log_p, loss_kl = kl_loss, loss_mse = mse_loss), dist



In [44]:
Regressor = LatentModel(2,4,
                       p_drop = 0,
                        hidden_dim = 64,
                        latent_dim = 16,
                       n_decoder_layer = 3,
                       norm = 'batch',
                       context_in_target=False
                       )

In [45]:
#train data loader
hparamas = dict(num_context = 15,
               num_extra_target = 16,
               batch_size = 400,
               context_in_target = False)
train_df = NutrientsDataset(data_train,hparamas['num_context'],hparamas['num_extra_target'])

train_loader = DataLoader(train_df,
                          batch_size=hparamas['batch_size'],
                         shuffle = True,
                         collate_fn=collate_fns(
                             hparamas['num_context'],hparamas['num_extra_target'], True,hparamas['context_in_target']))

In [46]:
# #eval data loader
# hparamas = dict(num_context = 15,
#                num_extra_target = 16,
#                batch_size = 40,
#                context_in_target = False)
# eval_df = NutrientsDataset(data_test,hparamas['num_context'],hparamas['num_extra_target'])

# train_loader = DataLoader(eval_df,
#                           batch_size=hparamas['batch_size'],
#                          shuffle = True,
#                          collate_fn=collate_fns(
#                              hparamas['num_context'],hparamas['num_extra_target'], True,hparamas['context_in_target']))

In [47]:
# data_train.iloc[:,:2]

In [61]:
#eval loss
def test(do_eval=True):
    """Run model on test/val data"""
    if do_eval:
        Regressor.eval()
    with torch.no_grad():
        target_x, target_y = data_test.iloc[:,:2], data_test.iloc[:,2:]
        context_x, context_y = data_train.iloc[:,:2], data_train.iloc[:,2:]

        context_x = torch.from_numpy(context_x.values).float()[None, :]
        context_y = torch.from_numpy(context_y.values).float()[None, :]
        target_x = torch.from_numpy(target_x.values).float()[None, :]
        target_y = torch.from_numpy(target_y.values).float()[None, :]
        print(context_x.shape, context_y.shape, target_x.shape, target_y.shape)
        y_pred, losses, extra = Regressor.forward(context_x, context_y, target_x, target_y)
        print(y_pred.shape)
    yr=(target_y-y_pred)[0].detach().cpu().numpy()
#     print(yr)
    return yr, y_pred, losses, extra 

In [62]:
test()[0]

torch.Size([1, 20837, 2]) torch.Size([1, 20837, 4]) torch.Size([1, 5209, 2]) torch.Size([1, 5209, 4])
torch.Size([1, 5209, 4])


array([[-0.19728503, -0.3132522 ,  0.845472  , -0.02644168],
       [-0.35095876, -0.42687303,  0.5910084 ,  0.02158483],
       [-0.4037054 , -0.18509154,  0.25507063,  0.09816705],
       ...,
       [-0.1871854 , -0.2815249 ,  0.92175674, -0.00298213],
       [-0.2412051 , -0.41691044,  0.9135716 ,  0.06411079],
       [-0.22108693, -0.25960636,  0.94111425, -0.04580018]],
      dtype=float32)

In [51]:
opt = torch.optim.Adam(Regressor.parameters(), lr=1e-4)

In [63]:
from tqdm.auto import tqdm 

for epoch in range(1000):
    loss = 0 
    mse_loss = 0
    Regressor.train()
    for batch in tqdm(train_loader):
        context_x, context_y, target_x, target_y = batch
#         context_x.shape
        Regressor.zero_grad()
        y_pred, losses, extra = Regressor.forward(context_x, context_y, target_x, target_y)
#         print(y_pred.shape)
        losses['loss'].backward()
        loss += losses['loss'].cpu().detach().numpy()
        mse_loss+=losses['loss_mse'].cpu().detach().numpy()
        opt.step()
    loss /= len(train_loader)
    
    print(epoch)
    print('ELBO train_loss', loss)
    print('mse train_loss', mse_loss/len(train_loader))
    
    val_loss = test()[0]
    val_loss = np.mean(np.abs(val_loss))
    print('val_loss = ', val_loss)
    print("-----------------------------------------------------------------------")

  0%|          | 0/53 [00:00<?, ?it/s]

0
ELBO train_loss 0.40834776804132283
mse train_loss 0.0067994619350669515
torch.Size([1, 20837, 2]) torch.Size([1, 20837, 4]) torch.Size([1, 5209, 2]) torch.Size([1, 5209, 4])
torch.Size([1, 5209, 4])
val_loss =  0.15735935
-----------------------------------------------------------------------


  0%|          | 0/53 [00:00<?, ?it/s]

1
ELBO train_loss 0.24481830478839153
mse train_loss 0.002267636929831977
torch.Size([1, 20837, 2]) torch.Size([1, 20837, 4]) torch.Size([1, 5209, 2]) torch.Size([1, 5209, 4])
torch.Size([1, 5209, 4])
val_loss =  0.12186013
-----------------------------------------------------------------------


  0%|          | 0/53 [00:00<?, ?it/s]

...

  0%|          | 0/53 [00:00<?, ?it/s]

717
ELBO train_loss -3.263999507112323
mse train_loss 0.00037628159674877335
torch.Size([1, 20837, 2]) torch.Size([1, 20837, 4]) torch.Size([1, 5209, 2]) torch.Size([1, 5209, 4])
torch.Size([1, 5209, 4])
val_loss =  0.021006078
-----------------------------------------------------------------------


  0%|          | 0/53 [00:00<?, ?it/s]

718
ELBO train_loss -3.2425110497564638
mse train_loss 0.0003764521092273084
torch.Size([1, 20837, 2]) torch.Size([1, 20837, 4]) torch.Size([1, 5209, 2]) torch.Size([1, 5209, 4])


KeyboardInterrupt: 

# Meuse data testing

In [ ]:
%%capture
!pip install geopandas

In [ ]:
import os,requests
import geopandas as gpd

In [ ]:
allow_columns = ['lead', 'elev', 'dist', 'x', 'y', 'ffreq', 'soil']
label ='lead'

In [ ]:
meuse = gpd.read_file('meuse')
meuse.crs = {'init':'epsg:28992'}
meuse['x'] = meuse['geometry'].apply(lambda x: x.x)
meuse['y'] = meuse['geometry'].apply(lambda x: x.y)
meuse.sample(2)

In [ ]:
for col in ['ffreq', 'soil']:
    meuse[col] = pd.to_numeric(meuse[col]) * 1.0
meuse[allow_columns].info()

In [ ]:
np.random.seed(0)
test_indexes = np.random.choice(a=meuse.index, size=int(np.round(len(meuse.index.values)/4)))
train_indexes = [index for index in meuse.index if index not in test_indexes]
meuse_test = meuse.loc[test_indexes,:].copy()
meuse_train = meuse.loc[train_indexes,:].copy()
print('Number of observations in training: {}, in test: {}'.format(len(meuse_train), len(meuse_test)))

In [ ]:
df_train = meuse_train[allow_columns].copy()
norm_mean = meuse[allow_columns].mean()
norm_std = meuse[allow_columns].std()

hparams = dict(
    num_context=15,
    num_extra_target=16,
    batch_size=40,
    context_in_target=False,
)


In [ ]:
class DataSet(torch.utils.data.Dataset):
    def __init__(self, df, num_context=40, num_extra_target=10, label_names=['lead']):
        self.df = df
        self.num_context = num_context
        self.num_extra_target = num_extra_target
        self.label_names = label_names

    def get_rows(self, i):
        rows = self.df.iloc[i : i + (self.num_context + self.num_extra_target)].copy()

        # make sure tstp, which is our x axis, is the first value

        # This will be the last row, and will change it upon sample to let the model know some points are in the future

        x = rows.drop(columns=self.label_names).copy()
        y = rows[self.label_names].copy()
#         print(x.shape)
#         print(y.shape)
        return x, y


    def __getitem__(self, i):
        x, y = self.get_rows(i)
        return x.values, y.values
        
    def __len__(self):
        return len(self.df) - (self.num_context + self.num_extra_target)


In [ ]:
for i, batch in enumerate(loader_train):
    a,b,c,d = batch
    print(d.shape)

In [ ]:
df_train = meuse_train[allow_columns].copy()
df_train -= norm_mean
df_train /= norm_std

data_train = DataSet(
    df_train, hparams["num_context"], hparams["num_extra_target"]
)
loader_train = torch.utils.data.DataLoader(
    data_train,
    batch_size=hparams["batch_size"],
    shuffle=True,
    collate_fn=collate_fns(
        hparams["num_context"], hparams["num_extra_target"], sample=True, context_in_target=hparams["context_in_target"]
    ),
)

In [ ]:
df_test = meuse_test[allow_columns].copy()
df_test -= norm_mean
df_test /= norm_std

data_test = DataSet(
    df_test, hparams["num_context"], hparams["num_extra_target"]
)
loader_test = torch.utils.data.DataLoader(
    data_test,
    batch_size=hparams["batch_size"],
    shuffle=False,
    collate_fn=collate_fns(
        hparams["num_context"], hparams["num_extra_target"], sample=False, context_in_target=hparams["context_in_target"]
    ),
)

In [ ]:
Regressor = LatentModel(len(allow_columns)-1,1,
                       p_drop = 0.5,
                        hidden_dim = 64,
                        latent_dim = 16,
                       n_decoder_layer = 3,
                       norm = 'batch',
                       context_in_target=False
                       )

In [ ]:
from tqdm.auto import tqdm 

for epoch in range(1000):
    loss = 0 
    Regressor.train()
    for batch in tqdm(loader_train):
        context_x, context_y, target_x, target_y = batch
        print(context_x.shape)
        Regressor.zero_grad()
        y_pred, losses, extra = Regressor.forward(context_x, context_y, target_x, target_y)

        losses['loss'].backward()
        loss += losses['loss'].cpu().detach().numpy()
        opt.step()
    loss /= len(loader_train)
    
    print(epoch)
    print('train_loss', loss)